### Задание 1

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
# from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
# from implicit.evaluation import train_test_split
# from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('./input/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3
threshold = data['week_no'] < data['week_no'].max() - test_size_weeks

data_train = data[threshold]
data_test = data[~threshold]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id']
    weights = items_weights['weights']
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [14]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].median().reset_index()
total = sum(items_weights['sales_value'])
items_weights['weights'] = items_weights['sales_value'] / total
iw = items_weights[['item_id', 'weights']]

result['weighted_random_recommendation'] = \
    result['user_id'].apply(lambda x: weighted_random_recommendation(iw, n=5))

Wall time: 1.68 s


In [15]:
result.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2250409, 966779, 837760, 1080911, 9368200]","[9677395, 7432861, 6961519, 9194033, 1031594]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[845300, 36185, 914884, 1101544, 12487473]","[1018659, 17214981, 867158, 1079762, 949180]"


### Задание 2
Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах? - 
это значит, что мы должны не просто забивать товары не из ТОП-5000 одним индексом 999999 и учиться на всех данных.
А учиться только на взаимодействиях с товарами из ТОП5000

Если я правильно понимаю, то хороший бейзлайн - это бейзлайн будет давать высокие метрики без применения доп затрат.
По этому если популярные товары находятся сейчас "на волне", то при прогнозировании бейзлайн будет давать более высокие метрики, что поспсобствует созда
